imports

In [1]:
import requests
import pandas as pd
from itertools import product
import time
from google.cloud import bigquery
from google.oauth2 import service_account

upload json key

In [2]:
from google.colab import files
uploaded = files.upload()

Saving xq-data-bigquery-8d4927262daf.json to xq-data-bigquery-8d4927262daf.json


set destination

In [3]:
key_path = "xq-data-bigquery-8d4927262daf.json"
credentials = service_account.Credentials.from_service_account_file(key_path)
project_id = "xq-data-bigquery"

dataset_id = "staging_naep"
table_id = "grade_12_mathematics"
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

Base URL and shared parameters

In [4]:
url = "https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx"
shared_params = {
    "type": "data",
    "subject": "mathematics",
    "grade": "12",
    "year": "2019",
    "jurisdiction": "NP"
}

Parameter lists

In [5]:
subscales = ["MWPCM", "MWPS1", "MWPS2", "MWPS3", "MWPS4"]
stattype_list = ["MN:MN", "ALD:BA", "ALD:PR", "ALD:AD"]
variables = ["B018101", "B035202", "C102803", "C102806", "C102807", "C102809", "M831501", "M831504", "M831505", "M831506", "M831901", "M831902", "M831903", "M831905", "M831906", "M834102", "T044401", "T077310", "T112501", "T112502", "T126026", "T133801", "T136902", "T136903", "T136904", "T136905", "TNSTUDM", "TTIME4M"]

Collect data

In [ ]:
all_results = []

for subscale, stattype, variable in product(subscales, stattype_list, variables):
    params = shared_params.copy()
    params.update({
        "subscale": subscale,
        "stattype": stattype,
        "variable": variable
    })

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        result_json = response.json()

        # Tag metadata in each result
        for record in result_json.get("result", []):
            record.update({
                "subscale_requested": subscale,
                "stattype_requested": stattype,
                "variable_requested": variable
            })
            all_results.append(record)

        print(f"✅ Success: {subscale} | {stattype} | {variable}")
        time.sleep(0.5)  # Optional: be polite to the server

    except Exception as e:
        print(f"❌ Failed: {subscale} | {stattype} | {variable} | Error: {str(e)}")

Convert to DataFrame

In [ ]:
df = pd.DataFrame(all_results)

Upload job

In [ ]:
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",
    autodetect=True
)

job = client.load_table_from_dataframe(df, full_table_id, job_config=job_config)

job.result()

LoadJob<project=xq-data-bigquery, location=US, id=fae5c51f-2d38-41d7-9aeb-4cccd03ef4a6>